# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pickle
from IPython.display import display
from surprise import Dataset
from surprise import KNNBasic
from surprise import Reader

# Initializing Parameters

In [ ]:
mu= 4.6818430002382145
rating = pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 2/Baseline/Train_data.csv") # Empty Rating Matrix
bi =np.load("/content/drive/MyDrive/RS Data/Assignment 2/Baseline/bi.npy")
bu =np.load("/content/drive/MyDrive/RS Data/Assignment 2/Baseline/bi.npy")

In [ ]:
with open('/content/drive/MyDrive/RS Data/Assignment 2/Baseline/recipe_names.pkl', 'rb') as fp:
    names = pickle.load(fp)
    print('Recipe_Name dictionary imported successfully')
    

Recipe_Name dictionary imported successfully


# Multi Round Recommendation System (Baseline Estimate)

In [ ]:
class Multi_RecSys():
  def __init__(self,train,mu,bi,bu,names):
    self.train = train
    self.mu = mu
    self.bi = bi
    self.bu = bu
    self.R_ = self.mu + np.dot(self.bu,self.bi.T)
    self.dct = {user_id: index for index, user_id in enumerate(self.train.index)}
    self.rec_name = names
  
  def BaseLine(self):

    user_id = int(input("Enter Your User Id:  "))
    os.system('cls')
    print("\n\n")
    
    print(f"Welcome User {user_id}")
    print("\n\n")
    
    uid = self.dct[user_id]
    unrated_items = np.where(self.train.iloc[uid, :] == 0)[0]

    # Sort the predicted ratings for unrated items in descending order
    sorted_ratings = np.argsort(self.R_[uid, unrated_items])[::-1]

    # Recommend the top N items to the user
    N = 5
    recommended_items = unrated_items[sorted_ratings][:N]
    print(f" We have these recommendations for you today: \n ")
    new_dict = {}
    for idx in recommended_items:
      key = list(self.rec_name.keys())[idx]
      value = self.rec_name[key]
      new_dict[key] = value

    output_df = pd.DataFrame.from_dict(new_dict, orient='index', columns=['Recpie_Name'])
    output_df = output_df.rename_axis('Recipe_ID')
    display(output_df)
    return recommended_items

  def get_recommendations(self):
    print("This system works on Baseline Estimated Recommendation System")
    self.BaseLine()

  def KNN(self):
    reader = Reader(rating_scale=(0, 5))
    data = Dataset.load_from_df(pd.DataFrame(self.train), reader)
    sim_options = {'name': 'cosine', 'user_based': False}
    k = 3
    model = KNNBasic(k=k, sim_options=sim_options)
    trainset = data.build_full_trainset()
    model.fit(trainset)
    item_similarity_matrix = model.compute_similarities()
    return item_similarity_matrix,model,trainset

  def multi_round(self,file_path):
    rounds=1
    while rounds != 10:
      if rounds == 1:
        self.get_recommendations()
        cnt = int(input("Do you want further recommendations? (1/0)  "))
        if cnt == 0:
          inp = int(input("Give us the recipe_id that you like: "))
          print("info")
          break
        
      else:
        if rounds == 2:
          item_similarity_matrix,model,trainset = self.KNN(file_path)
        inp = int(input("Give us the recipe_id that you prefer: "))
        # Rank items based on similarity to item of interest
        item_of_interest = str(inp)
        item_id = model.trainset.to_inner_iid(item_of_interest)
        similar_items = model.get_neighbors(item_id, k=k+1)

        # Choose a similarity threshold
        similarity_threshold = 0.5

        # Recommend items based on similarity and user's previous ratings
        recommended_items = [model.trainset.to_raw_iid(item) for item in similar_items if item != item_id and item_similarity_matrix[item_id, item] >= similarity_threshold and trainset.ur[trainset.to_inner_iid('user1')] == 0]
        print("\n")
        print(recommended_items)
        print("\n \n")
        cnt = int(input("Do you want further recommendations? (1/0)  "))
        if cnt == 0:
          inp = int(input("Give us the recipe_id that you like: "))
          print("info")
          break
        
      rounds+=1

In [ ]:
MRR=Multi_RecSys(rating,mu,bi,bu,names)
MRR.multiround()

Note: This approach requries a bit more ram due to which it crashes sometimes on collab otherwise it works well.